In [1]:
# import dependencies
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt    

In [2]:
# read in csv
mouse_file = "../../Downloads/WorkingDirectory/gt-virt-data-pt-12-2021-u-c/05-Matplotlib/Homework/Instructions/Pymaceuticals/data/Mouse_metadata.csv"
study_file = "../../Downloads/WorkingDirectory/gt-virt-data-pt-12-2021-u-c/05-Matplotlib/Homework/Instructions/Pymaceuticals/data/Study_results.csv"

mouse_master = pd.read_csv(mouse_file)
study_master = pd.read_csv(study_file)


In [26]:
# view mouse columns
mouse_master.columns

Index(['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)'], dtype='object')

In [27]:
# view mouse data
mouse_master.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [28]:
# view study columns
study_master.columns

Index(['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites'], dtype='object')

In [29]:
# view study data
study_master.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [30]:
mouse_master.count()

Mouse ID        249
Drug Regimen    249
Sex             249
Age_months      249
Weight (g)      249
dtype: int64

In [31]:
# count number unique mice in the mouse file
mouse_master["Mouse ID"].nunique()

249

In [32]:
study_master.count()

Mouse ID              1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [33]:
# count number unique mice in the study file
study_master["Mouse ID"].nunique()

249

In [11]:
# count null values in each column

# null["null count"] = study_master["Timepoint"].isna().sum()
# null

# def null_cnt(df_input, column_input, column_output):
#     df_input[column_output] = df_input[column_input].isna().sum()
#     return df_input[column_output]

# null_cnt(study_master, "Timepoint", "null")

In [12]:
#calculate the number of null values for each column
print("The number of null values:")

def print_null_msg(df_input, textdf, textfeature, column_output):
    return print(f"{textdf} -- {textfeature}: {df_input[textfeature].isna().sum().sum()}")

print_null_msg(study_master, "study_master", "Timepoint", "null count")
print_null_msg(study_master, "study_master", "Tumor Volume (mm3)", "null count")
print_null_msg(study_master, "study_master", "Metastatic Sites", "null count")
print_null_msg(mouse_master, "mouse_master", "Drug Regimen", "null count")
print_null_msg(mouse_master, "mouse_master", "Sex", "null count")
print_null_msg(mouse_master, "mouse_master", "Age_months", "null count")
print_null_msg(mouse_master, "mouse_master", "Weight (g)", "null count")

The number of null values:
study_master -- Timepoint: 0
study_master -- Tumor Volume (mm3): 0
study_master -- Metastatic Sites: 0
mouse_master -- Drug Regimen: 0
mouse_master -- Sex: 0
mouse_master -- Age_months: 0
mouse_master -- Weight (g): 0


In [47]:
# left merge study to mouse_master, mouse_master, how = "left", on = "Mouse ID")
mouse_combined = pd.merge(mouse_master, study_master, how = "outer", on = "Mouse ID")
mouse_combined.count()

Mouse ID              1893
Drug Regimen          1893
Sex                   1893
Age_months            1893
Weight (g)            1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [42]:
# calculate the number of mice
unique_mice = mouse_combined["Mouse ID"].nunique()
unique_cnt_df = pd.DataFrame([unique_mice]).rename(columns = {0: "Unique Mice"})
unique_cnt_df

,Unique Mice
0,249


In [129]:
# id duplicate Mouse ID at Timepoint level
#mouse_combined["tm_str"] = mouse_combined["Timepoint"].astype(str)
mouse_combined["idt_key"] = mouse_combined["Mouse ID"] + (mouse_combined["Timepoint"].astype(str))
keys = mouse_combined["idt_key"].value_counts()
keys_df = pd.DataFrame(keys)
keys_df = keys_df.rename(columns = {"idt_key": "cnt"})
dupkeys_df = keys_df.loc[keys_df["cnt"] > 1, :]
dupkeys_df.reset_index(inplace = True)
dupkeys_df = dupkeys_df.rename(columns = {"index": "idt_key"})
dupkeys_df

,idt_key,cnt
0,g98920,2
1,g98910,2
2,g9895,2
3,g98915,2
4,g9890,2


In [133]:
# get all of the data for the duplicate mice, creating a dataframe
duplicate_mouseid = pd.merge(dupkeys_df, mouse_combined, how = "left", on = "idt_key")

,idt_key,cnt,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,dupkey,tm_str,duplicat count,duplicate count
0,g98920,2,g989,Propriva,Female,21,26,20,54.657650,1,g98920,20,NaN,NaN
1,g98920,2,g989,Propriva,Female,21,26,20,55.326122,1,g98920,20,NaN,NaN
2,g98910,2,g989,Propriva,Female,21,26,10,51.745156,0,g98910,10,NaN,NaN
3,g98910,2,g989,Propriva,Female,21,26,10,49.880528,0,g98910,10,NaN,NaN
4,g9895,2,g989,Propriva,Female,21,26,5,48.786801,0,g9895,5,NaN,NaN
5,g9895,2,g989,Propriva,Female,21,26,5,47.570392,0,g9895,5,NaN,NaN
6,g98915,2,g989,Propriva,Female,21,26,15,51.325852,1,g98915,15,NaN,NaN
7,g98915,2,g989,Propriva,Female,21,26,15,53.442020,0,g98915,15,NaN,NaN
8,g9890,2,g989,Propriva,Female,21,26,0,45.000000,0,g9890,0,NaN,NaN
9,g9890,2,g989,Propriva,Female,21,26,0,45.000000,0,g9890,0,NaN,NaN


In [138]:
# create a dataframe without the duplicates
nondups_df = keys_df.loc[keys_df["cnt"] == 1, :]
nondups_df.reset_index(inplace = True)
nondups_df = nondups_df.rename(columns = {"index": "idt_key"})
nondups_df = pd.merge(nondups_df, mouse_combined, how = "left", on = "idt_key")
nondups_df

,idt_key,cnt,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,dupkey,tm_str,duplicat count,duplicate count
0,a20330,1,a203,Infubinol,Female,20,23,30,59.523197,1,a20330,30,NaN,NaN
1,q6335,1,q633,Zoniferol,Male,12,25,5,47.183802,0,q6335,5,NaN,NaN
2,q61010,1,q610,Ramicane,Female,18,21,10,44.991485,0,q61010,10,NaN,NaN
3,q6100,1,q610,Ramicane,Female,18,21,0,45.000000,0,q6100,0,NaN,NaN
4,q61035,1,q610,Ramicane,Female,18,21,35,36.561652,2,q61035,35,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1878,i33425,1,i334,Ramicane,Female,8,20,25,38.248817,1,i33425,25,NaN,NaN
1879,i33430,1,i334,Ramicane,Female,8,20,30,39.005554,1,i33430,30,NaN,NaN
1880,i33435,1,i334,Ramicane,Female,8,20,35,39.769552,2,i33435,35,NaN,NaN
1881,i33440,1,i334,Ramicane,Female,8,20,40,40.442370,2,i33440,40,NaN,NaN


In [143]:
# count unique mice in clean df
unique_mice_cnt = nondups_df["Mouse ID"].nunique()
unique_mice_cnt_df = pd.DataFrame([unique_mice_cnt]).rename(columns = {0: "Unique Mouse Count"})
unique_mice_cnt_df

,Unique Mouse Count
0,249
